In [1]:
import pandas as pd
fedrate = pd.read_csv("../.Database/fedrate.csv") # https://www.kaggle.com/datasets/federalreserve/interest-rates
loan = pd.read_csv("../.Database/cleaned_loan.csv")
unemp = pd.read_csv("../.Database/us_state_unemployment_rate.csv") # https://www.bls.gov/
pd.set_option('display.max_columns', None)

In [2]:
wanted_columns = ['Year','Month','Effective Federal Funds Rate']
fedrate = fedrate[wanted_columns]

fedrate = fedrate.dropna(subset=['Effective Federal Funds Rate'])
# Renaming multiple columns
fedrate.rename(columns={
    'Year': 'issue_y',
    'Month': 'issue_m'
}, inplace=True)
fedrate.head()


,issue_y,issue_m,Effective Federal Funds Rate
0,1954,7,0.80
1,1954,8,1.22
2,1954,9,1.06
3,1954,10,0.85
4,1954,11,0.83


In [3]:
loan['issue_d'] = pd.to_datetime(loan['issue_d'], format='%b-%y') + pd.offsets.MonthBegin(0)
loan['issue_y'] = loan['issue_d'].dt.year
loan['issue_m'] = loan['issue_d'].dt.month

In [4]:
merged_df = loan.merge(fedrate, on=['issue_y', 'issue_m'], how='left')
merged_df.head()

,annual_inc,addr_state,delinq_2yrs,dti,emp_title,emp_length,funded_amnt,grade,home_ownership,inq_last_6mths,installment,int_rate,issue_d,loan_amnt,loan_status,mths_since_last_delinq,pub_rec,purpose,sub_grade,term,title,total_acc,total_pymnt,acc_now_delinq,log_annual_inc,loan_status_grouped,issue_y,issue_m,Effective Federal Funds Rate
0,24000.0,AZ,0.0,27.65,NaN,10+ years,5000,B,RENT,1.0,162.87,10.65,2011-12-01,5000,Fully Paid,0.0,0.0,credit_card,B2,36 months,Computer,9.0,5861.071414,0.0,10.085809,0.0,2011,12,0.07
1,30000.0,GA,0.0,1.00,Ryder,< 1 year,2500,C,RENT,5.0,59.83,15.27,2011-12-01,2500,Charged Off,0.0,0.0,car,C4,60 months,bike,4.0,1008.710000,0.0,10.308953,1.0,2011,12,0.07
2,12252.0,IL,0.0,8.72,NaN,10+ years,2400,C,RENT,2.0,84.33,15.96,2011-12-01,2400,Fully Paid,0.0,0.0,small_business,C5,36 months,real estate business,10.0,3003.653644,0.0,9.413444,0.0,2011,12,0.07
3,49200.0,CA,0.0,20.00,AIR RESOURCES BOARD,10+ years,10000,C,RENT,1.0,339.31,13.49,2011-12-01,10000,Fully Paid,35.0,0.0,other,C1,36 months,personel,37.0,12226.302210,0.0,10.803649,0.0,2011,12,0.07
4,36000.0,AZ,0.0,11.20,Veolia Transportaton,3 years,5000,A,RENT,3.0,156.46,7.90,2011-12-01,5000,Fully Paid,0.0,0.0,wedding,A4,36 months,My wedding loan I promise to pay back,12.0,5631.377753,0.0,10.491274,0.0,2011,12,0.07


In [5]:
unemp.rename(columns={
    'state_code': 'addr_state',
    'year': 'issue_y',
    'month': 'issue_m',
    'value': 'state_unemployment' 
}, inplace=True)

unemp.head()

,addr_state,issue_y,issue_m,state_unemployment
0,AL,2015,12,6.1
1,AL,2015,11,6.1
2,AL,2015,10,6.1
3,AL,2015,9,6.1
4,AL,2015,8,6.2


In [6]:
len(unemp['addr_state'].value_counts())

51

In [7]:
merged_df = merged_df.merge(unemp, on=['issue_y', 'issue_m','addr_state'], how='left')
merged_df

,annual_inc,addr_state,delinq_2yrs,dti,emp_title,emp_length,funded_amnt,grade,home_ownership,inq_last_6mths,installment,int_rate,issue_d,loan_amnt,loan_status,mths_since_last_delinq,pub_rec,purpose,sub_grade,term,title,total_acc,total_pymnt,acc_now_delinq,log_annual_inc,loan_status_grouped,issue_y,issue_m,Effective Federal Funds Rate,state_unemployment
0,24000.0,AZ,0.0,27.65,NaN,10+ years,5000,B,RENT,1.0,162.87,10.65,2011-12-01,5000,Fully Paid,0.0,0.0,credit_card,B2,36 months,Computer,9.0,5861.071414,0.0,10.085809,0.0,2011,12,0.07,8.8
1,30000.0,GA,0.0,1.00,Ryder,< 1 year,2500,C,RENT,5.0,59.83,15.27,2011-12-01,2500,Charged Off,0.0,0.0,car,C4,60 months,bike,4.0,1008.710000,0.0,10.308953,1.0,2011,12,0.07,9.6
2,12252.0,IL,0.0,8.72,NaN,10+ years,2400,C,RENT,2.0,84.33,15.96,2011-12-01,2400,Fully Paid,0.0,0.0,small_business,C5,36 months,real estate business,10.0,3003.653644,0.0,9.413444,0.0,2011,12,0.07,9.5
3,49200.0,CA,0.0,20.00,AIR RESOURCES BOARD,10+ years,10000,C,RENT,1.0,339.31,13.49,2011-12-01,10000,Fully Paid,35.0,0.0,other,C1,36 months,personel,37.0,12226.302210,0.0,10.803649,0.0,2011,12,0.07,11.3
4,36000.0,AZ,0.0,11.20,Veolia Transportaton,3 years,5000,A,RENT,3.0,156.46,7.90,2011-12-01,5000,Fully Paid,0.0,0.0,wedding,A4,36 months,My wedding loan I promise to pay back,12.0,5631.377753,0.0,10.491274,0.0,2011,12,0.07,8.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
257772,63078.0,PA,0.0,31.70,customer service,10+ years,17000,D,MORTGAGE,0.0,413.32,15.99,2015-01-01,17000,Fully Paid,0.0,0.0,debt_consolidation,D2,60 months,Debt consolidation,28.0,17196.520000,0.0,11.052127,0.0,2015,1,0.11,5.5
257773,48000.0,CO,0.0,36.93,supervisor,10+ years,4200,D,MORTGAGE,0.0,147.64,15.99,2015-01-01,4200,Charged Off,38.0,0.0,medical,D2,36 months,Medical expenses,45.0,1026.020000,0.0,10.778956,1.0,2015,1,0.11,4.1
257774,54000.0,FL,1.0,13.22,Coordinator of RSVP,< 1 year,10775,A,RENT,0.0,327.95,6.03,2015-01-01,10775,Fully Paid,16.0,0.0,debt_consolidation,A1,36 months,Debt consolidation,21.0,11071.870000,0.0,10.896739,0.0,2015,1,0.11,5.9
257775,27000.0,FL,0.0,18.58,Painter,2 years,6225,D,RENT,1.0,220.37,16.49,2015-01-01,6225,Fully Paid,0.0,0.0,debt_consolidation,D3,36 months,Debt consolidation,4.0,7050.460000,0.0,10.203592,0.0,2015,1,0.11,5.9


In [35]:
merged_df.to_csv("../.Database/merged_cleaned_loan.csv", index=False)